In [1]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score, KFold,StratifiedKFold,train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import roc_auc_score


from subprocess import check_output
print(check_output(["ls", "/home/mucahit/PycharmProjects/school/industrialdataworks/weekly_reports/data/titanic"]).decode("utf8"))

train_data = pd.read_csv('/home/mucahit/PycharmProjects/school/industrialdataworks/weekly_reports/data/titanic/train.csv', dtype={'Age': np.float16})
test_data = pd.read_csv('/home/mucahit/PycharmProjects/school/industrialdataworks/weekly_reports/data/titanic/test.csv', dtype={'Age': np.float16})
test2 = pd.read_csv("/home/mucahit/PycharmProjects/school/industrialdataworks/weekly_reports/data/titanic/test.csv" ,  dtype={'Age': np.float16})

train_data.head()

gender_submission.csv
test.csv
train.csv



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
print('Train columns with null values:\n', train_data.isnull().sum())
print("-" * 10)

Train columns with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------


In [3]:
dropping = ['PassengerId', 'Ticket','Cabin','Fare']
train_data.drop(dropping,axis=1, inplace=True)
test_data.drop(dropping,axis=1, inplace=True)
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,S


In [4]:
data = [train_data,test_data]
for dataset in data:
    dataset['Embarked'].fillna(dataset['Embarked'].mode()[0],inplace=True)
    Embarked = np.zeros(len(dataset))
    Embarked[dataset['Embarked']== 'C'] = 1
    Embarked[dataset['Embarked']== 'Q'] = 2
    Embarked[dataset['Embarked']== 'S'] = 3
    dataset['Embarked'] = Embarked

In [5]:
train_data['Age'].fillna(train_data['Age'].median(), inplace = True)
test_data['Age'].fillna(test_data['Age'].median(), inplace = True)

In [6]:
print('Train columns with null values:\n', train_data.isnull().sum())
print("-" * 10)
print('Test columns with null values:\n', test_data.isnull().sum())
print("-" * 10)

Train columns with null values:
 Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64
----------
Test columns with null values:
 Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Embarked    0
dtype: int64
----------


In [7]:
data = [train_data,test_data]
for dataset in data:
    dataset['Age'] = dataset['Age']
    dataset.loc[ dataset['Age'] <= 11, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 11) & (dataset['Age'] <= 18), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 18) & (dataset['Age'] <= 22), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 22) & (dataset['Age'] <= 27), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 27) & (dataset['Age'] <= 33), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 33) & (dataset['Age'] <= 40), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 66), 'Age'] = 6
    dataset.loc[ dataset['Age'] > 66, 'Age'] = 6

train_data['Age'].value_counts()


4.0    303
6.0    150
3.0    106
5.0    101
2.0     92
1.0     71
0.0     68
Name: Age, dtype: int64

In [8]:
data = [train_data,test_data]
titles = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}

for dataset in data:
    # extract titles
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
    # replace titles with a more common title or as Rare
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr',\
                                            'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    # convert titles into numbers
    dataset['Title'] = dataset['Title'].map(titles)
    # filling NaN with 0, to get safe
    dataset['Title'] = dataset['Title'].fillna(0)
train_data = train_data.drop(['Name'], axis=1)
test_data = test_data.drop(['Name'], axis=1)

In [9]:
data = [train_data,test_data]
for dataset in data:
    sex = np.zeros(len(dataset))
    sex[dataset['Sex']== 'male'] = 1
    sex[dataset['Sex']== 'female'] = 0
    dataset['Sex'] = sex

In [10]:
data = [train_data,test_data]
for dataset in data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1    

In [11]:
data = [train_data,test_data]
for dataset in data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

In [12]:
train_data = train_data.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_data = test_data.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)

In [13]:
train_data.head()

,Survived,Pclass,Sex,Age,Embarked,Title,IsAlone
0,0,3,1.0,2.0,3.0,1,0
1,1,1,0.0,5.0,1.0,3,0
2,1,3,0.0,3.0,3.0,2,1
3,1,1,0.0,5.0,3.0,3,0
4,0,3,1.0,5.0,3.0,1,1


In [14]:
train_y=train_data['Survived']
train_ft=train_data.drop('Survived',axis=1)
kf = StratifiedKFold(n_splits=10)
print(train_ft.head())
print(train_y.head())

   Pclass  Sex  Age  Embarked  Title  IsAlone
0       3  1.0  2.0       3.0      1        0
1       1  0.0  5.0       1.0      3        0
2       3  0.0  3.0       3.0      2        1
3       1  0.0  5.0       3.0      3        0
4       3  1.0  5.0       3.0      1        1
0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64


In [15]:
from sklearn.svm import SVC, LinearSVC
svc = SVC(C = 45, gamma = 0.03)
svc.fit(train_ft, train_y) 

acc_SVM = cross_val_score(svc,train_ft,train_y,cv=kf)
print(acc_SVM.mean())

0.8136531040744523


In [16]:
from sklearn.neighbors.nearest_centroid import NearestCentroid
NN = NearestCentroid()
NN.fit(train_ft, train_y)
acc_NN = cross_val_score(NN,train_ft,train_y,cv=kf)
print(acc_NN.mean())

0.7711931108841221


In [17]:
logreg = LogisticRegression(C=1e3, solver='liblinear', multi_class='auto')
logreg.fit(train_ft, train_y)
acc_logreg = cross_val_score(logreg,train_ft,train_y,cv=kf)
print(acc_logreg.mean())

0.8047520145272955


In [18]:
knn = KNeighborsClassifier(n_neighbors = 3) 
knn.fit(train_ft, train_y)  
acc_knn = cross_val_score(knn,train_ft,train_y,cv=kf)
print(acc_knn.mean())

0.7856500397230735


In [19]:
gaussian = GaussianNB() 
gaussian.fit(train_ft, train_y) 
acc_gss = cross_val_score(gaussian,train_ft,train_y,cv=kf)
print(acc_gss.mean())

0.7800312109862672


In [20]:
linear_svc = SVC(C=100, kernel="rbf")
linear_svc.fit(train_ft, train_y) 
acc_svc = cross_val_score(linear_svc,train_ft,train_y,cv=kf)
print(acc_svc.mean())

0.8002701168993303


In [21]:
predictions = svc.predict(test_data)
print(predictions)

[0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1
 1 0 0 0 0 0 1 1 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 1 1 0 1 0
 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 1 1 0 0 1 1 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 1 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0
 1 0 0 0 0 0 1 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0
 0 1 0 1 1 1 0 1 0 0 1]


In [24]:
submission = pd.DataFrame({ 'PassengerId': test2['PassengerId'],
                            'Survived': predictions })
submission.to_csv("/home/mucahit/Desktop/submission.csv", index=False)